In [10]:
from tqdm.notebook import tqdm
from IPython.display import display, HTML

import numpy as np
import pandas as pd

import plotly.graph_objects as go

import datetime
from pathlib import Path

import intake

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras import backend as K

from tensorflow.keras.layers import ConvLSTM2D, Conv2D, Conv3D, Dense, TimeDistributed, MaxPooling2D, GlobalAveragePooling2D 
from tensorflow.keras.layers import Activation, Dropout, Flatten, BatchNormalization, Input, LSTM

In [11]:
catalog = intake.open_catalog(Path('../catalog.yml'))
source = getattr(catalog, 'treesat_multi')
df = source.read()

In [12]:
selected_bands = [f'B{x}' for x in range(2, 9)] + ['B8A', 'B11', 'B12', 'TCI_R', 'TCI_G', 'TCI_B']

In [13]:
target = source.metadata['categories']['multi'] # multi / trinary

labels = df[target].to_numpy()

model_dir = Path('models').joinpath('seasons')
model_dir.mkdir(parents=True, exist_ok=True)

data_dir = Path('seasonal_median')

In [5]:
tf.keras.utils.get_custom_objects().clear()
@tf.keras.utils.register_keras_serializable(name='f1_majority')
def f1_majority(y_true, y_pred):
    # Convert multilabel proportions to single-label majority class
    majority_class = (y_true == tf.keras.ops.amax(y_true, keepdims=True, axis=1))
    majority_class = tf.cast(majority_class, tf.float32)
    
    true_positives = K.sum(K.round(majority_class * y_pred))
    possible_positives = K.sum(K.round(majority_class))

    recall = true_positives / (possible_positives + K.epsilon())
    
    predicted_positives = K.sum(K.round(y_pred))

    precision = true_positives / (predicted_positives + K.epsilon())
    
    return 2 * precision * recall / (precision + recall + K.epsilon())

class KerasModelCreator:
    def crop_y(self, y):
        lower = 0.1
        bounded_y = np.where(y < lower, 0.0, y)
        rescaled_y = bounded_y/bounded_y.sum(axis=1, keepdims=1)
        return rescaled_y
        
    def normalise_X(self, X, p=1):
        upper = np.percentile(X, 100-p)
        lower = np.percentile(X, p)
    
        bounded_X = np.where(X > upper, np.median(X), X)
        bounded_X = np.where(X < lower, np.median(X), bounded_X)
        
        scaled_X = (bounded_X - lower)/(upper - lower)
        return scaled_X
        
    def split_and_normalise(self, y, X, random_state=42, test_size=0.1):
        """Split and max scale."""
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state, shuffle=True)

        y_train, y_test = self.crop_y(y_train), self.crop_y(y_test)

        for i in range(X_train.shape[-1]):
            X_train[...,i] = self.normalise_X(X_train[...,i])
            X_test[...,i] = self.normalise_X(X_test[...,i])
        
        return X_train, X_test, y_train, y_test

    def build_model(self, y_train, input_shape, metrics):
        filter_base= 32
        dropout_base = 0.25
        
        m = tf.keras.Sequential()

        m.add(Input((None, *input_shape[2:])))
        
        for kernel_size in [3]:
            m.add(Conv2D(
                filters=filter_base, kernel_size=kernel_size, 
                padding='same', activation='relu'
            ))
            m.add(BatchNormalization())
            m.add(Dropout(dropout_base))
        
        m.add(Flatten())

        for filter_scale in [1]:
            m.add(Dense(filter_scale*filter_base, activation='relu'))
        
        m.add(Dense(
            y_train.shape[1], 
            activation='softmax', 
        ))

        loss = tf.keras.losses.Huber(
            delta=1.0,
            reduction='sum_over_batch_size'
        )

        opt = tf.keras.optimizers.Adam(
            learning_rate=0.001
        )
        m.compile(
            optimizer=opt,
            loss=loss,
            metrics=metrics
        )
        return m

        
    def run(self, X, y, model_path, epochs=10, overwrite=False):
        ''' 
        If not overwrite and there's an existing model, the model will 
        continue training if the given epoch is bigger than the previous,
        else just evaluate.
        Ensure train splits are the same across continuations / evaluations
        by not modifying the random_state in split_and_normalise.
        '''
        model_savepoint = model_path.parent.joinpath(model_path.stem)
        log_file = model_path.with_suffix('.log')

        if overwrite:
            for f in [model_path, log_file] + list(model_savepoint.glob('*')):
                f.unlink(missing_ok=True)

        X_train, X_test, y_train, y_test = self.split_and_normalise(y, X, random_state=42)
        
        default_metrics = ['accuracy', 'root_mean_squared_error', 'mean_squared_error', 'r2_score']
        
        if model_path.is_file():
            model = tf.keras.models.load_model(model_path)
        else:
            model = self.build_model(y_train, X_train.shape, default_metrics + [f1_majority])

        callbacks = [
            tf.keras.callbacks.BackupAndRestore(
                model_savepoint, save_freq='epoch', delete_checkpoint=False
            ),
            tf.keras.callbacks.CSVLogger(
                log_file, append=True
            )
        ]

        if log_file.is_file():
            df = pd.read_csv(log_file)[['epoch', 'loss'] + default_metrics + ['f1_majority']]
            df['epoch'] += 1
            print('Previous training:')
                
            display(HTML(df.to_html(index=False)))
        
        model.fit(
            X_train, y_train, epochs=epochs, verbose=1, batch_size=4, callbacks=callbacks, 
            shuffle=True, validation_data=(X_test, y_test)
        )

        model.save(model_path)
        
        return model

In [6]:
test_years = [2018]
train_years = [2017]
all_seasons = [3, 6, 9, 12]

In [9]:
selected_data = []
for year in train_years:
    for season in all_seasons:
        filepath = data_dir.joinpath(f'processed_treesat_{year}{str(season).zfill(2)}.npy')
        with open(filepath, 'rb') as f:
            data = np.load(f)
        selected_data.append(data)

# train_features = np.concatenate(selected_data)
# train_labels = np.tile(labels, (len(selected_data), 1))

train_features = np.stack(selected_data, axis=1)

print((None, *train_features.shape[2:]))

model_name = f'conv_seasons_{"_".join(map(str, all_seasons))}_years_{"_".join(map(str, train_years))}.keras'
model_path = model_dir.joinpath(model_name)
result = KerasModelCreator().run(
    train_features, labels, model_path, epochs=10, overwrite=True)

result

(None, 6, 6, 13)


ValueError: Kernel shape must have the same length as input, but received kernel of shape (3, 3, 13, 32) and input of shape (None, None, 6, 6, 13).